In [1]:
%matplotlib inline
import pandas as pd
from scrape.graph import get_merged_summaries_with_final_actual_intensities, create_graph_images, generate_boxplot_ci, generate_boxplot_ci_error, generate_boxplot_ci_error_for_days, generate_plot_ci_lines, generate_boxplot_ci_error_per_hour

# Carbon intensity forecast tracking

How this repo operates, for national data:

1. Periodically scrape JSON data from the National Grid API
2. Convert to CSV
3. Summarise in a giant combined CSV
4. Plot graphs and calculate statistics.

Most operations are possible via the CLI: see the `python3 run.py ...` commands.

## 1. Scrape JSON data from the National Grid API

We'll use the existing repo folder `./data`. Subfolders will be created for each endpoint.

The supported endpoints are defined in `api.py`. For national data, we'll use the two endpoints `national_fw48h`, `national_pt24h`.

Run the following commands repeatedly every half-hour until you have gathered the data you want:
```
python run.py download --output_directory data --now --endpoint national_fw48h
python run.py download --output_directory data --now --endpoint national_pt24h
```

Gathering data as it is published, every half hour, is important because historical forecasts are overwritten/unavailable (hence this project).

## 2. Convert to CSV

```
python3 run.py wrangle --input_directory "data/national_fw48h" --endpoint "national_fw48h"
python3 run.py wrangle --input_directory "data/national_pt24h" --endpoint "national_pt24h"
```

This will create `.csv` files in the same directory with identical names as the `.json` files.

## 3. Summarise

```
python3 run.py summary --input_directory "data/national_fw48h" --output_directory "data" --endpoint "national_fw48h"
python3 run.py summary --input_directory "data/national_pt24h" --output_directory "data" --endpoint "national_pt24h"
```
This will create `summary_national_fw48h.csv` and `summary_national_pt24h.csv` files in the `data` directory which combine all the available CSVs into one file.

In [ ]:
# Example
df = pd.read_csv("./data/summary_national_fw48h.csv").head()
df

## 4. Plots and statistics

```
python3 run.py graph --input_directory "data"
```

In [2]:
# Example to plot individual plots
summaries_merged_df = get_merged_summaries_with_final_actual_intensities("data", filter="national")

In [ ]:
fig = generate_plot_ci_lines(summaries_merged_df, hours_of_data=24)

In [ ]:
fig = generate_boxplot_ci(summaries_merged_df, hours_of_data=24)

In [ ]:
fig = generate_boxplot_ci_error(summaries_merged_df, hours_of_data=24)

In [ ]:
fig = generate_boxplot_ci_error_for_days(summaries_merged_df, days=7)

In [ ]:
fig = generate_boxplot_ci_error_per_hour(summaries_merged_df, days=7)

In [ ]:
create_graph_images("data", hours_of_data=24, days=7)

Note these CIs assume an approximate Normal distribution, which we can see from the days' histograms is not true.

In [3]:
from scrape.graph import generate_combined_stats_dataframe, update_stats_history

In [4]:
stats_combined_df = generate_combined_stats_dataframe(
    summaries_merged_df, days=100
)
stats_combined_df

/Users/nick.powell/.virtualenvs/carbon-intensity-forecast-tracking/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nick.powell/.virtualenvs/carbon-intensity-forecast-tracking/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/nick.powell/.virtualenvs/carbon-intensity-forecast-tracking/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nick.powell/.virtualenvs/carbon-intensity-forecast-tracking/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


error, gCO2/kWh                                       \
           count            mean    std   sem 95% confidence interval   
2023-03-14   903          -14.27  10.89  0.36        (-14.98, -13.56)   
2023-03-15  3099           -4.92  22.33   0.4          (-5.71, -4.13)   
2023-03-16  4405          -11.13  26.91  0.41        (-11.93, -10.34)   
2023-03-17  4417          -11.55  36.93  0.56        (-12.64, -10.46)   
2023-03-18  4415            31.8  24.47  0.37          (31.07, 32.52)   
2023-03-19  3678           48.56  46.22  0.76          (47.07, 50.06)   
2023-03-20  1921           -49.5  23.34  0.53        (-50.54, -48.45)   
2023-03-21  1430          -15.41  14.57  0.39        (-16.17, -14.66)   
2023-03-22  2551            0.63  14.07  0.28            (0.08, 1.17)   
2023-03-23  3873           -0.26  18.08  0.29           (-0.83, 0.31)   
2023-03-24  3696           10.13   7.46  0.12           (9.89, 10.37)   
2023-03-25  3212           -7.07  24.25  0.43          (-7.91, -6.23)   
2023-03-26  2167           -6.44  18.76   0.4          (-7.23, -5.65)   
2023-03-27  3859            16.1   24.0  0.39          (15.34, 16.85)   
2023-03-28  3954          -24.61  18.13  0.29        (-25.18, -24.05)   
2023-03-29  3962           -5.85  16.48  0.26          (-6.36, -5.33)   
2023-03-30  2615           -4.89  22.01  0.43          (-5.73, -4.04)   
2023-04-03     1            -1.0    nan   nan              (nan, nan)   
2023-04-04  1046          -12.29  46.03  1.42          (-15.09, -9.5)   
2023-04-05  3140          -26.29   28.4  0.51        (-27.28, -25.29)   
2023-04-06  4141          -30.31  25.13  0.39        (-31.08, -29.55)   
2023-04-07  4158            -8.9  33.06  0.51           (-9.9, -7.89)   
2023-04-08   968          -21.18  11.32  0.36        (-21.89, -20.46)   

           percentage error                                       
                       mean    std   sem 95% confidence interval  
2023-03-14           -10.15    8.8  0.29         (-10.72, -9.57)  
2023-03-15            -0.76  16.88   0.3          (-1.35, -0.16)  
2023-03-16            -4.34  30.54  0.46          (-5.24, -3.44)  
2023-03-17            -0.08  29.48  0.44           (-0.95, 0.79)  
2023-03-18            22.16  21.62  0.33          (21.52, 22.79)  
2023-03-19            49.71  54.84   0.9          (47.93, 51.48)  
2023-03-20           -25.71   11.7  0.27        (-26.23, -25.19)  
2023-03-21           -14.38  13.69  0.36        (-15.09, -13.67)  
2023-03-22             4.24  22.89  0.45            (3.36, 5.13)  
2023-03-23             4.29   28.0  0.45            (3.41, 5.17)  
2023-03-24             17.2  12.36   0.2            (16.8, 17.6)  
2023-03-25            -0.61  21.85  0.39           (-1.37, 0.14)  
2023-03-26             -3.8  14.93  0.32          (-4.42, -3.17)  
2023-03-27            10.09  14.06  0.23           (9.64, 10.53)  
2023-03-28           -16.84  12.44   0.2        (-17.23, -16.45)  
2023-03-29            -2.26  17.56  0.28          (-2.81, -1.71)  
2023-03-30            -1.32  24.17  0.47           (-2.25, -0.4)  
2023-04-03            -0.58    nan   nan              (nan, nan)  
2023-04-04            -2.61   28.1  0.87           (-4.31, -0.9)  
2023-04-05           -11.87  13.65  0.24        (-12.35, -11.39)  
2023-04-06           -17.27  14.56  0.23        (-17.71, -16.82)  
2023-04-07            -3.78  19.14   0.3           (-4.37, -3.2)  
2023-04-08            -9.56   5.08  0.16          (-9.88, -9.23)

In [6]:
update_stats_history("data", stats_combined_df, name="national")

/Users/nick.powell/code/github/nmpowell/carbon-intensity-forecast-tracking/scrape/graph.py:641: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  df_stats = df_stats.reindex(union_index)


ValueError: cannot reindex on an axis with duplicate labels

In [7]:
df_stats = pd.read_csv("data/stats_history_national.csv", header=[0, 1], index_col=0)

In [8]:
df_stats

Unnamed: 1_level_0 error, gCO2/kWh               \
                        count            mean    std   sem   
2023-03-14              903.0          -14.27  10.89  0.36   
2023-03-15             3099.0           -4.92  22.33  0.40   
2023-03-16             4405.0          -11.13  26.91  0.41   
2023-03-17             4417.0          -11.55  36.93  0.56   
2023-03-18             4415.0           31.80  24.47  0.37   
2023-03-19             3678.0           48.56  46.22  0.76   
2023-03-20             1921.0          -49.50  23.34  0.53   
2023-03-21             1430.0          -15.41  14.57  0.39   
2023-03-22             2551.0            0.63  14.07  0.28   
2023-03-23             3873.0           -0.26  18.08  0.29   
2023-03-24             3696.0           10.13   7.46  0.12   
2023-03-25             3212.0           -7.07  24.25  0.43   
2023-03-26             2167.0           -6.44  18.76  0.40   
2023-03-27             3859.0           16.10  24.00  0.39   
2023-03-28             3954.0          -24.61  18.13  0.29   
2023-03-29             3962.0           -5.85  16.48  0.26   
2023-03-30             2615.0           -4.89  22.01  0.43   
2023-04-03                1.0           -1.00    NaN   NaN   
2023-04-04             1046.0          -12.29  46.03  1.42   
2023-04-05             3140.0          -26.29  28.40  0.51   
2023-04-06             4141.0          -30.31  25.13  0.39   
2023-04-07             4158.0           -8.90  33.06  0.51   
2023-04-08              968.0          -21.18  11.32  0.36   
2023-03-14                NaN          -14.27  10.89  0.36   
2023-03-15                NaN           -4.92  22.33  0.40   
2023-03-16                NaN          -11.13  26.91  0.41   
2023-03-17                NaN          -11.55  36.93  0.56   
2023-03-18                NaN           31.80  24.47  0.37   
2023-03-19                NaN           48.56  46.22  0.76   
2023-03-20                NaN          -49.50  23.34  0.53   
2023-03-21                NaN          -15.41  14.57  0.39   
2023-03-22                NaN            0.63  14.07  0.28   
2023-03-23                NaN           -0.26  18.08  0.29   
2023-03-24                NaN           10.13   7.46  0.12   
2023-03-25                NaN           -7.07  24.25  0.43   
2023-03-26                NaN           -6.44  18.76  0.40   
2023-03-27                NaN           16.10  24.00  0.39   
2023-03-28                NaN          -24.61  18.13  0.29   
2023-03-29                NaN           -5.85  16.48  0.26   
2023-03-30                NaN           -4.89  22.01  0.43   
2023-04-03                NaN           -1.00    NaN   NaN   
2023-04-04                NaN          -12.29  46.03  1.42   
2023-04-05                NaN          -26.29  28.40  0.51   
2023-04-06                NaN          -30.31  25.13  0.39   
2023-04-07                NaN           -8.90  33.06  0.51   
2023-04-08                NaN          -21.18  11.32  0.36   

                                   percentage error               \
           95% confidence interval             mean    std   sem   
2023-03-14        (-14.98, -13.56)           -10.15   8.80  0.29   
2023-03-15          (-5.71, -4.13)            -0.76  16.88  0.30   
2023-03-16        (-11.93, -10.34)            -4.34  30.54  0.46   
2023-03-17        (-12.64, -10.46)            -0.08  29.48  0.44   
2023-03-18          (31.07, 32.52)            22.16  21.62  0.33   
2023-03-19          (47.07, 50.06)            49.71  54.84  0.90   
2023-03-20        (-50.54, -48.45)           -25.71  11.70  0.27   
2023-03-21        (-16.17, -14.66)           -14.38  13.69  0.36   
2023-03-22            (0.08, 1.17)             4.24  22.89  0.45   
2023-03-23           (-0.83, 0.31)             4.29  28.00  0.45   
2023-03-24           (9.89, 10.37)            17.20  12.36  0.20   
2023-03-25          (-7.91, -6.23)            -0.61  21.85  0.39   
2023-03-26          (-7.23, -5.65)            -3.80  14.93  0.32   
2023-03-27    